In [45]:
import json
import numpy as np
import pandas as pd 
import seaborn as sns
from commands import COMMANDS
from matplotlib import pyplot as plt

sns.set_style('whitegrid')

In [46]:
data = pd.read_csv('../data/ad_music_proccessed.csv', index_col=0)
leads = pd.read_csv('../data/leads.csv', index_col=0)

In [47]:
fund_extraction_lead = leads.iloc[0]
fund_extraction_lead[fund_extraction_lead != 0]

cookie                                       32cb902aaf374867c9b0f23a9b0d6dfa
unique_user_ids             ['U486984', 'U734415', 'U557722', 'U734312', '...
num_users                                                                  20
target_artist                                                           [nan]
target_user                                                             [nan]
change_password                                                            20
change_withdrawal_method                                                   20
disable_mfa                                                                20
homepage                                                                   30
personal_details                                                           20
see_balance                                                                56
withdraw_balance                                                           20
Name: 0, dtype: object

In [48]:
ck_data = data[data.cookie.eq(fund_extraction_lead.cookie)]
ck_data = ck_data[ck_data.command.isin(COMMANDS.funds_extraction)]
user_command_count = ck_data.groupby('user_id')['command'].nunique()
qualified_user_ids = user_command_count[user_command_count.eq(4)].index
len(qualified_user_ids)

20

The cookie '32cb902aaf374867c9b0f23a9b0d6dfa' has 20 accounts linked to it,
and has used change_password, change_withdrawal_method, disable_mfa and withdraw_balance  20 times.
from the cell above we learn that each account has used all four commands, so they were all affected.

In [49]:
print(qualified_user_ids)

Index(['U019517', 'U124328', 'U145168', 'U204274', 'U224257', 'U269290',
       'U369562', 'U486984', 'U557722', 'U575442', 'U631730', 'U651065',
       'U734312', 'U734415', 'U748815', 'U750775', 'U770286', 'U880812',
       'U918199', 'U919424'],
      dtype='object', name='user_id')


In [50]:
# Save affected Accounts
out = qualified_user_ids.values.tolist()

try:
    with open("affected_users.json", "r") as f:
        out_file = json.load(f)
except Exception as e:
    out_file = {}

out_file['funds_extraction'] = out
if 'ato' in out_file.keys():
    out_file['ato'] = out_file['ato'] + out
else:
    out_file['ato'] = out
with open("affected_users.json", "w") as f:
    json.dump(out_file, f, indent=4)



In [51]:
total_loss = 0
for user in qualified_user_ids:
    user_data = data[data.user_id.eq(user)]
    user_data = user_data[user_data.cookie.eq('32cb902aaf374867c9b0f23a9b0d6dfa')]
    user_loss = user_data.account_balance.max() - user_data.account_balance.min()
    total_loss += user_loss
    print(f"USER {user} LOST: {user_loss:.2f}$")
print(f"TOTAL LOSS: {total_loss:.2f}$")

USER U019517 LOST: 4869.21$
USER U124328 LOST: 2784.82$
USER U145168 LOST: 9885.61$
USER U204274 LOST: 4695.47$
USER U224257 LOST: 13196.38$
USER U269290 LOST: 13665.74$
USER U369562 LOST: 7876.86$
USER U486984 LOST: 6991.91$
USER U557722 LOST: 10712.65$
USER U575442 LOST: 7271.71$
USER U631730 LOST: 2859.13$
USER U651065 LOST: 14922.02$
USER U734312 LOST: 7198.01$
USER U734415 LOST: 14267.71$
USER U748815 LOST: 3727.69$
USER U750775 LOST: 14279.95$
USER U770286 LOST: 2954.42$
USER U880812 LOST: 11766.07$
USER U918199 LOST: 10615.70$
USER U919424 LOST: 9340.27$
TOTAL LOSS: 173881.33$


Turns out each user has used all four commands with this cookie,
so these 20 users have been affected by the attack.
The total loss is 173,881$.


by applying this query to the entire data we can see other users who have used these four commands.


In [52]:
ck_data = data[data.cookie.eq(fund_extraction_lead.cookie)]
extract = data[data.command.isin(COMMANDS.funds_extraction)]
user_command_count = extract.groupby('user_id')['command'].nunique()
general_user_ids = user_command_count[user_command_count >= 3].index
len(general_user_ids)

23

In [53]:
new_leads = general_user_ids.difference(qualified_user_ids)
new_leads

Index(['U551869', 'U633407', 'U937385'], dtype='object', name='user_id')

U551869 and U633407 sessions' contain no evidence of new cookie, device or ip - i conclude they were unaffected.


U937385 is a new artist account, and has used a single cookie since the creation of his account. I assume he is unaffected (might be related to other attacks.)

I evalutated these accounts by looking at the dataframes below.

In [54]:
data[data.user_id.eq(new_leads[0])]

,timestamp,user_id,user_email,account_creation_date,subscription_status,account_tier,path,http_method,ip,org,...,domain_rarity,ip_c_class,device_age,cookie_age,country_age,asn_age,ua_age,browser_age,os_age,ip_age
71727,2024-07-01 17:38:59,U551869,michaeladams@hotmail.com,2021-03-16 00:00:00,paid,artist,/artist/A000751/song/S000048/add_comment,POST,68.0.0.19,Cox Communications Inc.,...,0.310417,68.0.0,111,356,75,243,710,710,710,243
71746,2024-07-01 17:50:00,U551869,michaeladams@hotmail.com,2021-03-16 00:00:00,paid,artist,/artist/A000889/song/S000120/add_comment,POST,68.0.0.19,Cox Communications Inc.,...,0.310417,68.0.0,111,356,75,243,710,710,710,243
71749,2024-07-01 17:50:38,U551869,michaeladams@hotmail.com,2021-03-16 00:00:00,paid,artist,/user/U551869/disable_mfa,POST,68.0.0.19,Cox Communications Inc.,...,0.310417,68.0.0,111,356,75,243,710,710,710,243
71756,2024-07-01 17:57:21,U551869,michaeladams@hotmail.com,2021-03-16 00:00:00,paid,artist,/artist/A000899/song/S000025/song_details,GET,68.0.0.19,Cox Communications Inc.,...,0.310417,68.0.0,111,356,75,243,710,710,710,243
71758,2024-07-01 17:59:22,U551869,michaeladams@hotmail.com,2021-03-16 00:00:00,paid,artist,/user/U551869/see_balance,GET,68.0.0.19,Cox Communications Inc.,...,0.310417,68.0.0,111,356,75,243,710,710,710,243
71760,2024-07-01 18:00:29,U551869,michaeladams@hotmail.com,2021-03-16 00:00:00,paid,artist,/homepage,POST,68.0.0.19,Cox Communications Inc.,...,0.310417,68.0.0,111,356,75,243,710,710,710,243
72546,2024-07-02 12:43:10,U551869,michaeladams@hotmail.com,2021-03-16 00:00:00,paid,artist,/artist/A000751/song/S000053/review,POST,68.0.0.19,Cox Communications Inc.,...,0.310417,68.0.0,130,375,94,262,729,729,729,262
72547,2024-07-02 12:45:01,U551869,michaeladams@hotmail.com,2021-03-16 00:00:00,paid,artist,/artist/A000751/song/S000053/skip,POST,68.0.0.19,Cox Communications Inc.,...,0.310417,68.0.0,130,375,94,262,729,729,729,262
72552,2024-07-02 12:48:29,U551869,michaeladams@hotmail.com,2021-03-16 00:00:00,paid,artist,/artist/A000751/subscribe,POST,68.0.0.19,Cox Communications Inc.,...,0.310417,68.0.0,130,375,94,262,729,729,729,262
72553,2024-07-02 12:48:41,U551869,michaeladams@hotmail.com,2021-03-16 00:00:00,paid,artist,/user/U551869/withdraw_balance,POST,68.0.0.19,Cox Communications Inc.,...,0.310417,68.0.0,130,375,94,262,729,729,729,262


In [55]:
data[data.user_id.eq(new_leads[1])]

,timestamp,user_id,user_email,account_creation_date,subscription_status,account_tier,path,http_method,ip,org,...,domain_rarity,ip_c_class,device_age,cookie_age,country_age,asn_age,ua_age,browser_age,os_age,ip_age
40567,2024-07-02 00:49:38,U633407,cummingsmichele@hotmail.com,2023-01-29 00:00:00,paid,artist,/artist_dashboard,POST,12.0.0.56,"AT&T Services, Inc.",...,0.310417,12.0.0,29,354,628,179,210,210,210,179
40568,2024-07-02 00:54:33,U633407,cummingsmichele@hotmail.com,2023-01-29 00:00:00,paid,artist,/send_to/U568046/send_message,POST,12.0.0.56,"AT&T Services, Inc.",...,0.310417,12.0.0,29,354,628,179,210,210,210,179
40569,2024-07-02 00:57:11,U633407,cummingsmichele@hotmail.com,2023-01-29 00:00:00,paid,artist,/artist/A000958/song/S000028/add_comment,POST,12.0.0.56,"AT&T Services, Inc.",...,0.310417,12.0.0,29,354,628,179,210,210,210,179
40570,2024-07-02 00:57:11,U633407,cummingsmichele@hotmail.com,2023-01-29 00:00:00,paid,artist,/user/U633407/withdraw_balance,POST,12.0.0.56,"AT&T Services, Inc.",...,0.310417,12.0.0,29,354,628,179,210,210,210,179
40571,2024-07-02 00:57:12,U633407,cummingsmichele@hotmail.com,2023-01-29 00:00:00,paid,artist,/user/U633407/see_balance,POST,12.0.0.56,"AT&T Services, Inc.",...,0.310417,12.0.0,29,354,628,179,210,210,210,179
40572,2024-07-02 00:59:33,U633407,cummingsmichele@hotmail.com,2023-01-29 00:00:00,paid,artist,/artist/A000958/subscribe,POST,12.0.0.56,"AT&T Services, Inc.",...,0.310417,12.0.0,29,354,628,179,210,210,210,179
40573,2024-07-02 01:00:30,U633407,cummingsmichele@hotmail.com,2023-01-29 00:00:00,paid,artist,/search,GET,12.0.0.56,"AT&T Services, Inc.",...,0.310417,12.0.0,29,354,628,179,210,210,210,179
40574,2024-07-02 01:01:24,U633407,cummingsmichele@hotmail.com,2023-01-29 00:00:00,paid,artist,/artist/A000958/unsubscribe,POST,12.0.0.56,"AT&T Services, Inc.",...,0.310417,12.0.0,29,354,628,179,210,210,210,179
40575,2024-07-02 01:04:59,U633407,cummingsmichele@hotmail.com,2023-01-29 00:00:00,paid,artist,/artist/A000958/song/S000146/skip,POST,12.0.0.56,"AT&T Services, Inc.",...,0.310417,12.0.0,29,354,628,179,210,210,210,179
40576,2024-07-02 01:05:39,U633407,cummingsmichele@hotmail.com,2023-01-29 00:00:00,paid,artist,/user/U633407/disable_mfa,POST,12.0.0.56,"AT&T Services, Inc.",...,0.310417,12.0.0,29,354,628,179,210,210,210,179


In [56]:
data[data.user_id.eq(new_leads[2])][:50]

,timestamp,user_id,user_email,account_creation_date,subscription_status,account_tier,path,http_method,ip,org,...,domain_rarity,ip_c_class,device_age,cookie_age,country_age,asn_age,ua_age,browser_age,os_age,ip_age
30969,2024-07-01 08:45:21,U937385,scottholloway@gmail.com,2024-07-01 08:45:21,paid,artist,/artist/A000620/song/S000089/play,POST,217.0.0.77,British Telecommunications PLC,...,0.324374,217.0.0,0,0,0,0,0,0,0,0
30978,2024-07-01 08:51:18,U937385,scottholloway@gmail.com,2024-07-01 08:45:21,paid,artist,/artist/A000620/song/S000089/review,POST,217.0.0.77,British Telecommunications PLC,...,0.324374,217.0.0,0,0,0,0,0,0,0,0
30981,2024-07-01 08:52:42,U937385,scottholloway@gmail.com,2024-07-01 08:45:21,paid,artist,/artist/A000620/song/S000089/add_comment,POST,217.0.0.77,British Telecommunications PLC,...,0.324374,217.0.0,0,0,0,0,0,0,0,0
30984,2024-07-01 08:53:59,U937385,scottholloway@gmail.com,2024-07-01 08:45:21,paid,artist,/send_to/U283833/send_message,POST,217.0.0.77,British Telecommunications PLC,...,0.324374,217.0.0,0,0,0,0,0,0,0,0
30989,2024-07-01 09:00:06,U937385,scottholloway@gmail.com,2024-07-01 08:45:21,paid,artist,/artist/A000620/subscribe,POST,217.0.0.77,British Telecommunications PLC,...,0.324374,217.0.0,0,0,0,0,0,0,0,0
30990,2024-07-01 09:00:13,U937385,scottholloway@gmail.com,2024-07-01 08:45:21,paid,artist,/send_to/U148321/send_message,POST,217.0.0.77,British Telecommunications PLC,...,0.324374,217.0.0,0,0,0,0,0,0,0,0
30994,2024-07-01 09:03:41,U937385,scottholloway@gmail.com,2024-07-01 08:45:21,paid,artist,/user/U937385/recommendations,GET,217.0.0.77,British Telecommunications PLC,...,0.324374,217.0.0,0,0,0,0,0,0,0,0
34443,2024-07-03 19:53:22,U937385,scottholloway@gmail.com,2024-07-01 08:45:21,paid,artist,/artist/A000263/unsubscribe,POST,217.0.0.77,British Telecommunications PLC,...,0.324374,217.0.0,0,0,0,0,0,0,0,0
34444,2024-07-03 19:53:57,U937385,scottholloway@gmail.com,2024-07-01 08:45:21,paid,artist,/artist_dashboard,POST,217.0.0.77,British Telecommunications PLC,...,0.324374,217.0.0,0,0,0,0,0,0,0,0
34445,2024-07-03 19:54:12,U937385,scottholloway@gmail.com,2024-07-01 08:45:21,paid,artist,/artist/A000382/song/S000008/play,POST,217.0.0.77,British Telecommunications PLC,...,0.324374,217.0.0,0,0,0,0,0,0,0,0
